<a href="https://colab.research.google.com/github/AlePol27/2022_ML_Earth_Env_Sci/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML final project

Text

*Source: bla bla bla <a href="">label</a>.*

## Part I:

## Part II:

# Title

## Part I:

## Part II: